### Работа со звуком в питоне

#### Автоматическое распознание речи (ASR = automatic speech recognition)

ASR - это отдельная большая область компьютерной лингвистики и NLP со своими особенностями. Распознание (и генерация) речи привлекали интерес исследователей еще в шестидесятых годах прошлого века; первые программы для распознания речи появились уже тогда. Сегодня ASR осуществляется почти исключительно с помощью нейронных сетей (генерация, разумеется, тоже). 

Можно выделить три возможных метода распознания речи:

- акустический фонетический подход (1967 год)
- распознание паттерна (1993 год)
- с использованием ИИ (сегодня)

До того, как ИИ стал популярным и достаточно мощным, самый очевидный подход предполагал, что существуют различные фонетические элементы в устной речи, причем каждый такой элемент обладает своими акустическими свойствами. В общем-то, это вроде бы верно, но проблема заключается в том, что у разных носителей эти свойства могут быть очень отличающимися. Соответственно, самые первые программы пытались анализировать спектр речи, вычленять нужные признаки и затем сегментировать речь по фонемам, приписывая каждой фонеме ярлык. 

Более популярный (и позднее придуманный) метод может быть двух видов: первый вид - это когда мы сравниваем услышанное с набором шаблонов произнесенных слов (template method) и пытаемся угадать, какой больше подходит; второй - с использованием классического машинного обучения (еще называют стохастический, или вероятностный - потому что любое МО основано на вычислении вероятностей). 

Современный подход с использованием нейронных сетей объединяет оба предыдущих подхода и еще подключает искусственный интеллект. Это уже совсем сложная технически задача, но если вкратце, то такой подход включает в себя следующие шаги:

1. Получение звукового сигнала
2. Извлечение признаков (нейронные сети самостоятельно умеют извлекать признаки)
3. Акустическое моделирование (сеть, грубо говоря, распознает фонемы)
4. Языковое и лексическое моделирование (фонемы => слова)

Современные модели ASR бывают общие (им неважно, чью речь они распознают) и натренированные на одного пользователя или группу пользователей. Первые - самые сложные (ввиду разницы в произношении разных людей), вторые попроще. Еще бывают модели ASR, которые умеют дообучаться: скорее всего, на вашем телефоне распознает речь именно такая модель. 

#### Базовые библиотеки для работы со звуком

Питон, разумеется, умеет работать со звуком. В первую очередь нужно установить библиотеку pyaudio (она пригодится для разных вещей):

    pip install pyaudio
    
Без проблем эта библиотека устанавливается только на Windows. На макбуке понадобится устанавливать ее дополнительно через homebrew:

    brew install portaudio
    pip install pyaudio
    
На линуксе (имейте в виду: колаб = линукс!) необходимо тоже отдельно установить кое-что через apt:

    apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg
    pip install pyaudio
    
В колабе обе эти команды можно выполнить с восклицательным знаком в обычной ячейке (apt работает так же, как pip). 

Воспроизводить звук можно с помощью библиотеки playsound:

In [ ]:
!pip install playsound

from playsound import playsound

fullpath = ... # важно: playsound не может проиграть файл по относительному пути
playsound(fullpath)

Можно конвертировать звуковые файлы из одного формата в другой:

In [ ]:
!pip install pydub

from pydub import AudioSegment

# пути для конвертации                                                                         
src = "audio.mp3"
dst = "audio.wav"

# mp3 => wav                                                           
sound = AudioSegment.from_mp3(src)
sound.export(dst, format="wav")

Другие примеры использования библиотеки pydub можно найти на ее [странице гитхаба](https://github.com/jiaaro/pydub). 

#### Генерация и распознание речи

Существует довольно большое количество готовых моделей генерации речи и ее распознания; мы с вами нейронные сети обучать, конечно, не будем, воспользуемся готовыми инструментами. Для генерации речи можно легко использовать Google Text-to-Speech:

In [ ]:
!pip install gtts

from gtts import gTTS

tts = gTTS('текст', lang='ru')
tts.save('audio.mp3')

gTTS поддерживает большое количество разных языков (и их вариантов), полный список можно посмотреть [тут](https://cloud.google.com/text-to-speech/docs/voices). Можно указать только первые две буквы (например, en), а можно уточнить, какую версию хочется, например, en-US - для американского варианта произношения. 

Для распознания речи существует библиотека SpeechRecognition, которая умеет работать с несколькими разными моделями. 

    pip install SpeechRecognition

Также может понадобиться установить дополнительные библиотеки типа Vosk или pocketsphinx (это тоже модели распознания речи). 

В колабе микрофон, к сожалению, использовать не получится (но в Windows из локальной среды разработки - запросто!), поэтому остается только складывать в колаб готовые записи для распознания. 

Так можно распознать готовый аудиофайл:

In [ ]:
import speech_recognition as sr

AUDIO_FILE = "audio.wav" # путь к файлу-источнику

r = sr.Recognizer()
with sr.AudioFile(AUDIO_FILE) as source:
    audio = r.record(source)  # читает файл в свой внутренний формат

try:
    print(r.recognize_google(audio, language='en')) 
except sr.UnknownValueError:
    print("Google Speech Recognition не справился")
except sr.RequestError as e:
    print(f"Ошибка запроса: {e}")

Speech Recognition поддерживает только wav, aiff и flac расширения (но мы можем конвертировать mp3 в wav...)

Если вы работаете локально с компьютера, на котором есть микрофон, можно заставить SR распознавать речь прямо с микрофона:

In [ ]:
import speech_recognition as sr

r = sr.Recognizer()
with sr.Microphone() as source:
    print("Я вас слушаю!")
    audio = r.listen(source, 5, 15) # ждать 5 сек, слушать 15 сек
    
# в audio окажется записанный (но не сохраненный) звук с микрофона
    
try:
    print(r.recognize_google(audio, language='ru')) 
except sr.UnknownValueError:
    print("Google Speech Recognition не справился")
except sr.RequestError as e:
    print(f"Ошибка запроса: {e}")

SR может и просто записывать файлы с микрофона (после того, как послушал):

In [ ]:
# write audio to a RAW file
with open("microphone-results.raw", "wb") as f:
    f.write(audio.get_raw_data())

# write audio to a WAV file
with open("microphone-results.wav", "wb") as f:
    f.write(audio.get_wav_data())

# write audio to an AIFF file
with open("microphone-results.aiff", "wb") as f:
    f.write(audio.get_aiff_data())

# write audio to a FLAC file
with open("microphone-results.flac", "wb") as f:
    f.write(audio.get_flac_data())

А вот и (примерный) список поддерживаемых языков:

    + Afrikaans af
    + Basque eu
    + Bulgarian bg
    + Catalan ca
    + Arabic (Egypt) ar-EG
    +? Arabic (Jordan) ar-JO
    + Arabic (Kuwait) ar-KW
    +? Arabic (Lebanon) ar-LB
    + Arabic (Qatar) ar-QA
    + Arabic (UAE) ar-AE
    .+ Arabic (Morocco) ar-MA
    .+ Arabic (Iraq) ar-IQ
    .+ Arabic (Algeria) ar-DZ
    .+ Arabic (Bahrain) ar-BH
    .+ Arabic (Lybia) ar-LY
    .+ Arabic (Oman) ar-OM
    .+ Arabic (Saudi Arabia) ar-SA
    .+ Arabic (Tunisia) ar-TN
    .+ Arabic (Yemen) ar-YE
    + Czech cs
    + Dutch nl-NL
    + English (Australia) en-AU
    +? English (Canada) en-CA
    + English (India) en-IN
    + English (New Zealand) en-NZ
    + English (South Africa) en-ZA
    + English(UK) en-GB
    + English(US) en-US
    + Finnish fi
    + French fr-FR
    + Galician gl
    + German de-DE
    + Hebrew he
    + Hungarian hu
    + Icelandic is
    + Italian it-IT
    + Indonesian id
    + Japanese ja
    + Korean ko
    + Latin la
    + Mandarin Chinese zh-CN
    + Traditional Taiwan zh-TW
    +? Simplified China zh-CN ?
    + Simplified Hong Kong zh-HK
    + Yue Chinese (Traditional Hong Kong) zh-yue
    + Malaysian ms-MY
    + Norwegian no-NO
    + Polish pl
    +? Pig Latin xx-piglatin
    + Portuguese pt-PT
    .+ Portuguese (brasil) pt-BR
    + Romanian ro-RO
    + Russian ru
    + Serbian sr-SP
    + Slovak sk
    + Spanish (Argentina) es-AR
    + Spanish(Bolivia) es-BO
    +? Spanish( Chile) es-CL
    +? Spanish (Colombia) es-CO
    +? Spanish(Costa Rica) es-CR
    + Spanish(Dominican Republic) es-DO
    + Spanish(Ecuador) es-EC
    + Spanish(El Salvador) es-SV
    + Spanish(Guatemala) es-GT
    + Spanish(Honduras) es-HN
    + Spanish(Mexico) es-MX
    + Spanish(Nicaragua) es-NI
    + Spanish(Panama) es-PA
    + Spanish(Paraguay) es-PY
    + Spanish(Peru) es-PE
    + Spanish(Puerto Rico) es-PR
    + Spanish(Spain) es-ES
    + Spanish(US) es-US
    + Spanish(Uruguay) es-UY
    + Spanish(Venezuela) es-VE
    + Swedish sv-SE
    + Turkish tr
    + Zulu zu

#### Работа с разметкой Praat, ELAN

Существует несколько таких библиотек, мы возьмем на вооружение две:

    pip install pympi-ling
    pip install praat-textgrids
    
Как можно работать с разметкой ELAN (.eaf - файлы):

In [ ]:
from pympi import Elan

doc = Elan.Ead(file_path='...') # открываем файл

# у нашей разметки есть некоторые атрибуты:
doc.tiers # уровни
doc.linguistic_types 
doc.languages
doc.timeslots

# Можно вручную добавлять аннотацию:
doc.add_annotation('имя уровня', start, end, value='что вписать')

doc.add_language(lang_id) # добавить язык
doc.add_linguistic_type(lingtype) # добавить лингвистический тип

doc.add_tier(ID, ling) # добавить уровень разметки

doc.extract(start, end) # нарезать файл: извлечет указанный кусочек как отдельный объект

doc.filter_annotations(tier, filtin='строки, которые нужно показывать', filtex='ненужные строки')

doc.get_annotation_data_for_tier(ID) # вывести все, что аннотировано на уровне

doc.get_annotation_data_after_time(...)
doc.get_annotation_data_at_time(...)
doc.get_annotation_data_before_time(...)
doc.get_annotation_data_between_times(...)

doc.get_full_time_interval(...)
doc.get_gaps_and_overlaps(tier1, tier2, maxlen=-1) - возвращает генератор

# Слить два уровня в один  новый
doc.merge_tiers(tiers, tiernew=None, gapt=0, sep='_', safe=False)

doc.remove_all_annotations_from_tier(id_tier, clean=True) # удалить разметку с уровня

doc.remove_annotation(id_tier, time, clean=True)

doc.remove_tier(id_tier, clean=True)

doc.rename_tier(id_from, id_to)

doc.shift_annotations(time) # сместить разметку по времени

doc.to_file(file_path, pretty=True) # сохранить файл

doc.to_textgrid(filtin=[], filtex=[], regex=False) # конвертировать в textgrid

Как можно работать с разметкой Praat (textgrid):

In [ ]:
import textgrids

grid = textgrids.TextGrid(path)
# например, распечатать разметку по уровню syllables - так же можно распечатывать ее на других уровнях (но не у всех есть dur, вероятно)
for syll in grid['syllables']:
    label = syll.text.transcode()
    print(f'"{label}";{syll.dur}')